## Import required libraries:

In [3]:
import pandas as pd

## Load the excel data:

In [5]:
ta = pd.read_excel('data/meta/Transit_Agencies_for_Visualization.xlsx', sheet_name='TC AgencyList')
upt = pd.read_excel('data/ntd/TS2.2TimeSeriesSysWideOpexpSvc_2.xlsx', sheet_name='UPT')
bus = pd.read_excel('data/ntd/TS2.1TimeSeriesOpExpSvcModeTOS_3.xlsx', sheet_name='UPT')

## Filter data:

In [17]:
# Remove missing NTD ID's
ta = ta.dropna(how='all')
upt = upt.dropna(subset=['NTD ID'])
bus = bus.dropna(subset=['NTD ID'])

# Filter bus data by required modes
modes = ['MB', 'RB', 'CB', 'TB']
bus_filtered = bus[bus['Mode'].isin(modes)]

## Clean the data

In [24]:
# Combine project ID's
ta['Project ID'] = ta['Project ID'].combine_first(ta['"Other" primary Project ID'])

# Drop unused columns
ta_clean = ta.drop(columns=['ShowIndividual', '"Other" primary Project ID', 'Primary UZA', 'UZA Name',
                            'Agency Name', 'Reporter Acronym'])
upt_clean = upt.drop(columns=['Last Report Year', 'Legacy NTD ID', 'Agency Name', 'Agency Status',
                              'Reporter Type', 'City', 'State', 'Census Year', 'Primary UZA Name', 
                              'UZA', 'UZA Area SQ Miles', 'UZA Population', '2017 Status'])
bus_clean = bus_filtered.drop(columns=['Last Report Year', 'Legacy NTD ID', 'Agency Name', 'Agency Status',
                              'Reporter Type', 'City', 'State', 'Census Year', 'UZA Name', 'Mode', 'Service',
                              'Mode Status', 'UZA', 'UZA Area SQ Miles', 'UZA Population', '2017 Status'])

## Merge the data

In [25]:
def ntd_merge(df, name):
    # Merge dataframes
    merge = pd.merge(ta_clean, df, how='left', on='NTD ID')
    group = merge.drop(columns=['NTD ID']).groupby('Project ID')
    
    # Sum, stack, and export to CSV
    stack = group.sum().stack()
    stack.to_csv('data/output/' + name + '.csv', index_label=['id', 'year'], header=[name])
    print('Dataset ' + name + ' written')
    
ntd_merge(upt_clean, 'upt')
ntd_merge(bus_clean, 'bus')

Dataset upt written
Dataset bus written
